In [1]:
"Timetable parsing"

from tramnetwork import *

lookup_day = "20250128"

network = TramNetwork()
network.load_day(lookup_day)

loaded routes.txt
loaded transfers.txt
loaded stops.txt
loaded trips.txt
loaded stop_times.txt
loaded calendar.txt
loaded calendar_dates.txt
Found 203 Tram Stops (20250128).          


In [3]:
import pygame
pygame.init()

pygame 2.0.1 (SDL 2.0.14, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


(7, 0)

In [ ]:
# read strava gpx

strava_gpx_path = "assets/strava-28012025.gpx"

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
from dateutil import parser as dateparser

tree = ET.parse(strava_gpx_path)
root = tree.getroot()

class GpsPoint:

    def __init__(self, coords: tuple[float, float], time: datetime, elevation: float):
        self.coords = coords
        self.time = time
        self.elevation = elevation

    @classmethod
    def from_trkpt_element(cls, trkpt: Element) -> "GpsPoint":
        coords = trkpt.get("lat"), trkpt.get("lon")
        coords = tuple(float(c) for c in coords)
        time = datetime.fromisoformat(trkpt[1].text[:-1]) + timedelta(hours=1)
        elevation = float(trkpt[0].text)
        return cls(coords, time, elevation)
    
    def __repr__(self):
        return f"{self.__class__.__name__}({self.coords!r}, {self.time!r}, {self.elevation!r})"

point_elements = root.find("{http://www.topografix.com/GPX/1/1}trk").find("{http://www.topografix.com/GPX/1/1}trkseg")
gps_points = [GpsPoint.from_trkpt_element(element) for element in point_elements]

# because the gps data itself is too many points to draw all individually (>33811), we
# only consider every 10th point
gps_points = [p for i, p in enumerate(gps_points) if i % 10 == 0 or i == len(gps_points) - 1]

gps_point_times = [point.time for point in gps_points]
gps_point_coords = [point.coords for point in gps_points]

print(f"Loaded {len(gps_points)} gps points from {strava_gpx_path!r}")

2025-01-28 18:46:17
Loaded 3383 gps points from 'assets/strava-28012025.gpx'


In [ ]:
# show animation of noel riding a specific path

path = TramPath.load("ant.2025-01-28--9-06-18.txt", network)

import pygame.display
import pygame.time
import pygame.draw
import pygame.image
import pygame.transform
import pygame.font
import time, bisect
from pygame.math import Vector2

draw_player_opt = True
draw_player_path_opt = False
draw_plan_opt = True
draw_plan_path_opt = True

def get_curr_time(path: TramPath, t: float) -> datetime:
    path_start_time = min(path.arrival_times[0], gps_point_times[0])
    path_end_time = max(path.arrival_times[-1], gps_point_times[-1])
    return path_start_time + (path_end_time - path_start_time) * t

screen_size = (2000 // 3, 2216 // 3)
top_left_coords = (47.459425, 8.437827)
bottom_right_coords = (47.310020, 8.637158)
background_image = pygame.image.load("assets/karte3.png")
player_image = pygame.image.load("assets/tsp-pin-yellow.png")
plan_image = pygame.image.load("assets/plan-dot.png")
player_size_factor = 0.15
plan_size_factor = 0.08
animation_seconds = (get_curr_time(path, 1) - get_curr_time(path, 0)).total_seconds() / 60 # 60

start_time = time.time()
player_size_xy = player_image.get_size()[1] / player_image.get_size()[0]
player_size = (round(screen_size[0] * player_size_factor), round(screen_size[0] * player_size_factor * player_size_xy))
background_image = pygame.transform.smoothscale(background_image, screen_size)
player_image = pygame.transform.smoothscale(player_image, player_size)
plan_size_xy = player_image.get_size()[1] / player_image.get_size()[0]
plan_size = (round(screen_size[0] * plan_size_factor), round(screen_size[0] * plan_size_factor * plan_size_xy))
plan_image = pygame.transform.smoothscale(plan_image, plan_size)
percent_reverse_animation = 0.05

pygame.font.init()
cascadia_code = pygame.font.Font("C:\\Windows\\Fonts\\CascadiaCode.ttf", 50)

def interpolate_coords(c1: tuple[float], c2: tuple[float], t: float) -> tuple[float]:
    delta = (c2[0] - c1[0], c2[1] - c1[1])
    return (c1[0] + delta[0] * t, c1[1] + delta[1] * t)

def get_curr_plan_coords(path: TramPath, t: float, curr_time: datetime):
    "get coord from path at t ([0, 1]) from path"
    path_coords = []
    
    for i, (stop, ari, dep, curr_tram) in enumerate(path.zip()):
        if curr_time >= ari:
            path_coords.append(stop.coords)

        if dep is None or (curr_time >= ari and curr_time <= dep):
            return stop.coords, path_coords
        
        if i + 1 < len(path):
            next_stop = path.stops[i + 1]
            next_ari = path.arrival_times[i + 1]
            next_dep = path.departure_times[i + 1]
            next_tram = path.transportation_names[i + 1]

            if curr_time >= dep and curr_time <= next_ari:
                tram_t = (curr_time - dep) / (next_ari - dep)
                return interpolate_coords(stop.coords, next_stop.coords, tram_t), path_coords
        
    return top_left_coords, path_coords

def get_curr_player_coords(t: float, curr_time: datetime):
    index = bisect.bisect(gps_point_times, curr_time)
    if index <= 1 or index == len(gps_points):
        return gps_points[min(index, len(gps_points) - 1)].coords, gps_point_coords[:index]
    gps_p1 = gps_points[index - 1]
    gps_p2 = gps_points[index]
    sub_t = (curr_time - gps_p1.time) / (gps_p2.time - gps_p1.time)
    return interpolate_coords(gps_p1.coords, gps_p2.coords, sub_t), gps_point_coords[:index]

def px_from_coords(coords: tuple[float]) -> Vector2:
    lat_delta = top_left_coords[0] - bottom_right_coords[0]
    lon_delta = bottom_right_coords[1] - top_left_coords[1]
    relative_x = (coords[1] - top_left_coords[1]) / lon_delta
    relative_y = 1 - (coords[0] - bottom_right_coords[0]) / lat_delta
    return Vector2(float(relative_x * screen_size[0]), float(relative_y * screen_size[1]))

player_pos = Vector2(0, 0)
plan_pos = Vector2(0, 0)
curr_ingame_time = None
player_path_pos = []
plan_path_pos = []

def update():
    global curr_ingame_time
    t = ((time.time() - start_time) / animation_seconds) % 1

    curr_ingame_time = get_curr_time(path, t)

    curr_plan_coord, plan_path_coords = get_curr_plan_coords(path, t, curr_ingame_time)
    plan_pos.update(px_from_coords(curr_plan_coord))
    plan_path_pos.clear()
    for coord in plan_path_coords:
        plan_path_pos.append(px_from_coords(coord))
    plan_path_pos.append(plan_pos)
    
    curr_player_coord, player_path_coords = get_curr_player_coords(t, curr_ingame_time)
    player_pos.update(px_from_coords(curr_player_coord))
    player_path_pos.clear()
    for coord in player_path_coords:
        player_path_pos.append(px_from_coords(coord))
    player_path_pos.append(player_pos)

def draw_network(screen: pygame.Surface):
    # draw background image (map)
    screen.blit(background_image, (0, 0))

    # draw stops
    # overlay = pygame.Surface(screen_size, pygame.SRCALPHA)
    # for stop in network.stops:
    #     screen_pos = px_from_coords(stop.coords)
    #     pygame.draw.circle(overlay, (0, 0, 255, 100), screen_pos, 5)
    # screen.blit(overlay, (0, 0))

def draw_path(screen: pygame.Surface, path, color):
    for i in range(len(path) - 1):
        pos1, pos2 = path[i:i + 2]
        pygame.draw.line(screen, color, pos1, pos2, 5)

def draw_plan(screen: pygame.Surface):
    offset_pos = (plan_pos.x - player_size[0] / 2, plan_pos.y - player_size[1])
    screen.blit(plan_image, offset_pos)

def draw_player(screen: pygame.Surface):
    offset_pos = (player_pos.x - player_size[0] / 2, player_pos.y - player_size[1])
    screen.blit(player_image, offset_pos)

def draw_time(screen: pygame.Surface):
    time_str = curr_ingame_time.strftime("%H:%M")
    text_surface = cascadia_code.render(time_str, True, (0, 0, 0))
    screen.blit(text_surface, (screen_size[0] / 50, screen_size[0] / 50))

def draw(screen: pygame.Surface):
    draw_network(screen)
    draw_time(screen)

    if draw_plan_path_opt:
        draw_path(screen, plan_path_pos, (0, 0, 255))

    if draw_player_path_opt:
        draw_path(screen, player_path_pos, (255, 255, 0))

    if draw_plan_opt:
        draw_plan(screen)

    if draw_player_opt:
        draw_player(screen)

screen = pygame.display.set_mode(screen_size, pygame.SRCALPHA, 32)
clock = pygame.time.Clock()

pygame.display.init()
pygame.display.set_caption("ZH Tram Animation")
pygame.display.update()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    update()
    screen.fill((255, 255, 255))
    draw(screen)
    pygame.display.update()
    clock.tick(60)

pygame.display.quit()